# Painter classification
## Project 1 
#### Author: Fan Wenxiao
#### School: Beijing Institute of Technology

First, we classify the problem. The problem of classification of painters is a process of supervising learning and a classification problem. So for such a problem, we used the more established model resnet-18 on the network for training. The final accuracy is stable at 80%.

Secondly, in this problem, there is a problem that the data set is too small, so we enhance the data set by means of random rotation and mirror flipping, and increase the size of the data set from 400 to 4000. This is conducive to the generalization of the model.

##### Load data that has been data-enhanced and convert it to a tensor.

In [12]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim
import numpy as np
import math
import torch.utils.model_zoo as model_zoo

data_transform = transforms.Compose([
        #transforms.Resize(299),
        #transforms.CenterCrop(299),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                    	     std=[0.5, 0.5, 0.5])
    ])
train_dataset = ImageFolder(root='/content/drive/My Drive/Colab Notebooks/artist_distribute_new',transform=data_transform)
trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=128, shuffle=True)

print(train_dataset.class_to_idx)
#print(len(train_dataset[0]))

{'canaletto': 0, 'claude_monet': 1, 'george_romney': 2, 'jmw_turner': 3, 'john_robert_cozens': 4, 'paul_cezanne': 5, 'paul_gauguin': 6, 'paul_sandby': 7, 'peter_paul_rubens': 8, 'rembrandt': 9, 'richard_wilson': 10}


##### Define the network structure.

In [0]:
__all__ = ['ResNet', 'resnet18_cbam', 'resnet34_cbam', 'resnet50_cbam', 'resnet101_cbam',
           'resnet152_cbam']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)

        self.ca = ChannelAttention(planes * 4)
        self.sa = SpatialAttention()

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.ca(out) * out
        out = self.sa(out) * out

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=11):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def resnet18_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    #model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet18'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model


def resnet34_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet34'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model


def resnet50_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet50'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model


def resnet101_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet101'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model


def resnet152_cbam(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        pretrained_state_dict = model_zoo.load_url(model_urls['resnet152'])
        now_state_dict        = model.state_dict()
        now_state_dict.update(pretrained_state_dict)
        model.load_state_dict(now_state_dict)
    return model



##### Define the loss function and optimizer.

In [14]:
# In[]
net = resnet18_cbam()
net = torch.load('/content/drive/My Drive/Colab Notebooks/res18_40_128_old.pkl')
net = net.cuda()
print(net)
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
optimizer = optim.Adam(net.parameters(), lr=0.001)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ca): ChannelAttention(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (max_pool): AdaptiveMaxPool2d(output_size=1)
        (fc1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu1): ReLU()
        (fc2): Conv2d(

##### Start training, one batch is 256, train 20 epochs.

In [16]:
print("Start Training...")
LOSS = []
num_epoch = 20
for epoch in range(num_epoch):
    # 我们用一个变量来记录每10个batch的平均loss
    loss10 = 0.0
    # 我们的dataloader派上了用场
    for i, data in enumerate(trainloader):
        inputs, labels = data
        #print(labels)
        optimizer.zero_grad()
        inputs = inputs.cuda()
        labels = labels.cuda()
        #print(torch.cuda.current_device())
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss10 += loss.item()
        if i % 10 == 9:
            print('[Epoch %d, Batch %5d] loss: %.3f' %
                  (epoch + 1, i + 1, loss10 / 10))
            LOSS.append(loss10/10)
            loss10 = 0.0

print("Done Training!")

Start Training...
[Epoch 1, Batch    10] loss: 0.061
[Epoch 1, Batch    20] loss: 0.068
[Epoch 2, Batch    10] loss: 0.069
[Epoch 2, Batch    20] loss: 0.092
[Epoch 3, Batch    10] loss: 0.086
[Epoch 3, Batch    20] loss: 0.062
[Epoch 4, Batch    10] loss: 0.078
[Epoch 4, Batch    20] loss: 0.087
[Epoch 5, Batch    10] loss: 0.087
[Epoch 5, Batch    20] loss: 0.084
[Epoch 6, Batch    10] loss: 0.081
[Epoch 6, Batch    20] loss: 0.059
[Epoch 7, Batch    10] loss: 0.083
[Epoch 7, Batch    20] loss: 0.062
[Epoch 8, Batch    10] loss: 0.067
[Epoch 8, Batch    20] loss: 0.073
[Epoch 9, Batch    10] loss: 0.089
[Epoch 9, Batch    20] loss: 0.057
[Epoch 10, Batch    10] loss: 0.070
[Epoch 10, Batch    20] loss: 0.078
[Epoch 11, Batch    10] loss: 0.097
[Epoch 11, Batch    20] loss: 0.075
[Epoch 12, Batch    10] loss: 0.080
[Epoch 12, Batch    20] loss: 0.079
[Epoch 13, Batch    10] loss: 0.055
[Epoch 13, Batch    20] loss: 0.066
[Epoch 14, Batch    10] loss: 0.081
[Epoch 14, Batch    20] loss

##### Evaluation

In [17]:
# In[eval]
summ = 0
for ii in range(100):
  test_dataset = ImageFolder(root='/content/drive/My Drive/Colab Notebooks/test_set',transform=data_transform)
  testloader = torch.utils.data.DataLoader(test_dataset,batch_size=46, shuffle=True)
  eval_loss = 0
  correct = 0
  total = 0
  for i, data in enumerate(testloader):
    inputs, labels = data
    #print(labels)
    
    inputs = inputs.cuda()
    labels = labels.cuda()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    eval_loss += loss.item()
    total += labels.size(0)
    pred = torch.max(outputs, 1)[1]
    print('pred: ',pred,' labels: ',labels)
    correct += (pred == labels).sum().item()
    summ += (100 * correct / total)
  print('eval_loss: ',eval_loss)
  print('correct: ',correct)
  print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
print("total_acc: ", summ/100)

pred:  tensor([ 9,  3,  9,  0,  9,  3,  1,  4,  4,  6, 10,  7, 10,  2,  4,  7,  3,  7,
         7,  5,  1,  8,  5,  3,  2,  2,  1,  3,  6,  2,  3,  5,  3,  4,  1,  2,
         3, 10,  1,  8,  2,  6,  9,  9,  6,  2], device='cuda:0')  labels:  tensor([ 9,  3,  3,  0,  9,  3,  1,  4,  4,  5, 10,  7,  3,  2,  4,  7,  3,  7,
         1,  5,  1,  8,  5, 10,  2,  2,  5,  3,  6,  2,  0,  1,  3,  4,  1,  2,
         3,  3,  1,  8,  2,  6,  9,  9,  6,  2], device='cuda:0')
eval_loss:  1.1506589651107788
correct:  37
Accuracy of the network on the test images: 80 %
pred:  tensor([ 1,  2,  9,  0,  3,  2,  2,  5,  9,  3,  3,  4,  1,  4,  2, 10,  6,  7,
         8,  3,  0,  4,  2,  3,  7,  1,  5,  1,  8,  2, 10,  5,  3,  9,  7,  7,
         6,  1,  8,  4,  2,  3,  3,  0,  9,  6], device='cuda:0')  labels:  tensor([ 1,  2,  9,  3,  3,  2,  2,  5,  9,  3,  3,  4,  1,  4,  2,  3,  6,  3,
         8,  0,  0,  4,  2,  3,  7,  1,  1,  1,  5,  2, 10,  5,  3,  9,  7,  7,
         6,  5,  8,  4,  2,  3, 10,

"\ntest_dataset = ImageFolder(root='/content/drive/My Drive/Colab Notebooks/test_set',transform=data_transform)\ntestloader = torch.utils.data.DataLoader(test_dataset,batch_size=46, shuffle=True)\neval_loss = 0\ncorrect = 0\ntotal = 0\nfor i, data in enumerate(testloader):\n    inputs, labels = data\n    #print(labels)\n    \n    inputs = inputs.cuda()\n    labels = labels.cuda()\n    outputs = net(inputs)\n    loss = criterion(outputs, labels)\n    eval_loss += loss.item()\n    total += labels.size(0)\n    pred = torch.max(outputs, 1)[1]\n    print('pred: ',pred,' labels: ',labels)\n    correct += (pred == labels).sum().item()\nprint('eval_loss: ',eval_loss)\nprint('correct: ',correct)\nprint('Accuracy of the network on the test images: %d %%' % (100 * correct / total))\n"

As you can see, the accuracy of the model is 81%.